In [1]:
# Import Packages
from html.parser import HTMLParser
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import ssl
import os

In [47]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Get links for all finals
url = 'https://issasports.com/results/champs08/evtindex.htm'
master_url = 'https://issasports.com/results/champs08/'
page = requests.get(url, verify=False)
soup = BeautifulSoup(page.text, 'html.parser')
links = []
a_tags = soup.find_all('a')
for tag in a_tags:
    if 'Prelims' not in tag.text and 'Semis' not in tag.text and 'Latest' not in tag.text:
        links.append(master_url + tag['href'])

print(len(links))

c:\Users\jcameron\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'issasports.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


95


In [ ]:
# Download pages based on links collected
count = 0
for link in links:
    try:
        page = requests.get(link, verify=False).text
        filepath = os.path.join(r'../champs/web_pages/2008', f'{count}.html')
        f = open(filepath, 'w')
        f.write(page)
        f.close()
        count += 1
    except:
        continue

In [49]:
# Loop through directory and rename files
directory = '../champs/web_pages/2008'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        try:
            with open(f) as page:
                soup = BeautifulSoup(page, 'html.parser')
                r = soup.find('pre').text
                r = r.splitlines()

                # Get event details
                event_details = r[5]
                event_details = event_details.split()
                if event_details[6].strip().lower() != 'class':
                    event = event_details[4] + " " + event_details[5].strip()+ " " + event_details[6].strip()
                else:
                    event = event_details[4] + " " + event_details[5].strip()
                cl = event_details[-1] if event_details[-1].isnumeric() else event_details[-2].strip()
                gender = event_details[2].strip()
                new_name = f'Class_{cl}_{gender}_{event}.html'
                new_name = os.path.join(directory, new_name).replace('/', '\\')
            os.rename(f, new_name)
        except:
            continue

In [21]:
# Get results data from web pages
from parse_data import get_races, get_field_events, get_relays
directory = '../champs/web_pages/2019'
clms = ['event', 'class', 'gender', 'position', 'name', 'school', 'mark', 'points']
df = pd.DataFrame(columns=clms)

# Loop through files and get results data
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)

    # Get Race results data from web pages
    if os.path.isfile(f) and ('Dash' in filename or 'Run' in filename or 'Hurdles' in filename ):
        with open(f) as page:
            data = get_races(page) # Imp function to parse event data
            frame = pd.DataFrame(data, columns=clms)
            df = df.append(frame) # Append each event to dataframe
    
    # Get Field Events results data from web pages
    if os.path.isfile(f) and ('Throw' in filename or 'Put' in filename or 'Jump' in filename ):
        with open(f) as page:
            data = get_field_events(page) # Imp function to parse event data
            frame = pd.DataFrame(data, columns=clms)
            df = df.append(frame) # Append each event to dataframe

     # Get Relay results data from web pages
    if os.path.isfile(f) and 'Relay' in filename:
        with open(f) as page:
            data = get_relays(page) # Imp function to parse event data
            frame = pd.DataFrame(data, columns=clms)
            df = df.append(frame) # Append each event to dataframe

# Create CSV file with results data
df.to_csv('../champs/csv_files' + '\\' + 'champs_2019.csv', index=False)

In [53]:
page = '../champs/web_pages/2022/87.html'
with open(page) as pg:

    soup = BeautifulSoup(pg, 'html.parser')
    r = soup.find('pre').text
    r = r.splitlines()
    lst = []

    # Get event details
    for line in r:
        if 'Event' in line.strip():
            event_details = line
            print(event_details)

    event_details = event_details.split()
    #print(event_details)
    if event_details[6].strip().lower() != 'class':
        event = event_details[4] + " " + event_details[5].strip()+ " " + event_details[6].strip()
    else:
        event = event_details[4] + " " + event_details[5].strip()
    print(event)
    cl = event_details[-1] if event_details[-1].isnumeric() else event_details[-2].strip()
    gender = event_details[2].strip()
    
    # Event Results
    for line in r:
        my_dict = {}
        if len(line) > 0:
            clean_line = line.split('  ')
            clean_line = list(filter(None, clean_line))
            #print(clean_line)
            if (clean_line[0][0].isnumeric() and clean_line[0][1] != ')') or (clean_line[0].strip()[0:2].isnumeric() and clean_line[0].strip()[2] != ')'):
                if (clean_line[0].strip()[2] != '.' and clean_line[0].strip()[1] != '.' ):
                    position = clean_line[0].strip().split(' ',1)[0]
                    school = clean_line[0].strip().split(' ',1)[1]

                    my_dict['event'] = event
                    my_dict['class'] = cl
                    my_dict['gender'] = gender
                    my_dict['position'] = position
                    my_dict['name'] = ''
                    my_dict['school'] = school
            
                    try:
                        if clean_line[-1].strip() != '' and clean_line[-1].strip().isnumeric():
                            points = clean_line[-1].strip()
                        elif clean_line[-1].strip() == '' and clean_line[-2].strip().isnumeric():
                            points = clean_line[-2].strip()
                        else:
                            points = 0
                        mark = clean_line[1].strip()
                        my_dict['mark'] = mark
                        my_dict['points'] = points
                    except IndexError:
                        points = 0
                        mark = clean_line[-2].strip()
                        my_dict['mark'] = mark
                        my_dict['points'] = points
                    lst.append(my_dict)
    #print(lst)

Event 87  Heptathlon: #7 Girls 13-19 800 Meter Run
                        Women - Team Rankings - 38 Events Scored             
- 38 Events
